# Exploring the 505 Runways that do NOT contain all information
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** October 26, 2021

***
These 505 records referencing Runways come from the 599 runway records within the 2142 unknown records of Standard Format Remarks.

## Import Statements

In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

## Read in Files

1. Dataset containing 599 records referencing runways
2. Dataset containing 94 records referencing runways with ALL relevant information included

In [2]:
# Read in the dataset containing all 599 runway reference remarks (NOT hit on by initial regex)
rwy599 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/std_rwyComplete_599.csv')

In [3]:
rwy599

,REMARKS,UAS_Location_Runways
0,(Via MOR) Aircraft observed a white UAS off th...,['RWY 7']
1,(via MOR) Aircraft observed a red UAS while W ...,['RWY 27R']
2,1836 Aircraft observed a UAS off the right sid...,"['RWY 10L', 'RWY 10R']"
3,Aircraft observed a silver quadcopter UAS fro...,['RWY 17R']
4,Aircraft observed a 1 foot wide UAS at 175 fee...,['RWY9L']
...,...,...
594,"Military helicopter - G20326/H60 - Salem, OR (...",['RWY 31']
595,MultipleAircraft reported an unauthorized UAS ...,['RWY28R']
596,Received MOR via email: Aircraft observed a UA...,['RWY 31']
597,VFR Aircraft observed a UAS off the left side ...,['RWY35']


In [4]:
# Read in the dataset containing all 94 runway reference remarks containing ALL necessary information (NOT hit on by initial regex)
rwy94 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/other2142_runways94_UAS_latlong.csv')

In [5]:
rwy94 = rwy94[['REMARKS', 'UAS_Location_Runways']]
rwy94

KeyError: "['UAS_Location_Runways'] not in index"

#### Obtain dataframe containing only 505 records referencing Runways that are still 'To Be Explored'

In [ ]:
rwy510 = rwy599[~rwy599['REMARKS'].isin(rwy94['REMARKS'])].dropna().drop_duplicates(subset=['REMARKS'])

In [ ]:
rwy510

In [ ]:
# Export final new_df to .csv 
rwy510.to_csv('rwy510.csv', index = False)

#### Import Output_All_Points.csv file from Lex and Merge with dataframe containing 509 runway records

In [ ]:
output_all_points = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Output_All_Points.csv',encoding='cp1252')

In [ ]:
rwy510_linked = pd.merge(rwy510, output_all_points, on = 'REMARKS', how = 'left')

In [ ]:
rwy510_linked = rwy510_linked[['REMARKS', 'UAS_Location_Runways', 'REPORTINGFACILITY', 'UASLOCATION', 'RWYLOCATION']]

In [ ]:
rwy510_linked

In [ ]:
# Export final new_df to .csv 
rwy510_linked.to_csv('rwy510_linked.csv', index = False)

#### Explore records of the 510 referencing runways that also have a valid REPORTINGFACILITY and UAS_Location_Runways

In [ ]:
rwy510_noNull = rwy510_linked.dropna().reset_index()

In [ ]:
rwy510_noNull

In [ ]:
# Export final new_df to .csv 
rwy510_noNull.to_csv('rwy118_one.csv', index = False)

In [ ]:
# Create List that contains each Standard Remark, and the Heading/Direction
# information contained in each remark

remark_uas_loc = []
remarks = rwy510_noNull['REMARKS']

# regular expression for any heading/direction
headir_regex = '\.?[0-9]\.?[0-9]*[0-9]*\s?mile?[s]?\s?final?'

# Loop through all remarks and search for the heading/direction regex above
for i in range(len(remarks)):
    head_dir = re.findall(headir_regex, remarks[i])
    remark_uas_loc.append(remarks[i])
    remark_uas_loc.append(head_dir)

# Split Remarks and Heading/Directions into two seperate lists and create
# pandas dataframe
remark = []
uas_loc = []

for i in range(0, len(remark_uas_loc), 2):
    remark.append(remark_uas_loc[i])
    uas_loc.append(remark_uas_loc[i+1])

remark_uas_loc_df = pd.DataFrame()
remark_uas_loc_df['REMARKS'] = remark
remark_uas_loc_df['UAS_Distance'] = uas_loc
#remark_uas_loc_df['UAS_Location_Runways'] = std_rwyComplete_MERGED['UAS_Location_Runways']

In [ ]:
remark_uas_loc_df

In [ ]:
# Export final new_df to .csv 
remark_uas_loc_df.to_csv('rwy118_two.csv', index = False)

In [ ]:
display(HTML(remark_uas_loc_df.to_html()))

In [ ]:
# Get list of UAS locations from the above dataframe
uas_dist = remark_uas_loc_df['UAS_Distance'] 

# If the regular expressions did not hit on any location information, pass it UNKN for now
for i in range(len(remark_uas_loc_df)):
    if len(uas_dist[i]) == 0:
        uas_dist[i] = 'UNKN'

In [ ]:
remark_uas_loc_df

In [ ]:
rwy_mileFinal = remark_uas_loc_df[remark_uas_loc_df['UAS_Distance']!= 'UNKN'].reset_index()
rwy_mileFinal

In [ ]:
rwy_mileFinal_complete = pd.merge(rwy_mileFinal, rwy510_noNull, on = 'REMARKS', how = 'left')
rwy_mileFinal_complete

In [ ]:
rwy_mile_finalDf = pd.DataFrame(rwy_mileFinal_complete)
rwy_mile_finalDf

In [ ]:
# Export final new_df to .csv 
rwy_mileFinal_complete.to_csv('rwy11_allInfo.csv', index = False)

#### Link to airports_cleaned_declination.csv

In [ ]:
airports_declination = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned_declination.csv')

In [ ]:
airports_declination

In [ ]:
rwy_mileFinal_complete.rename(columns = {'UASLOCATION' : 'IDENT'}, inplace = True)

In [ ]:
rwy_mileFinal_linked = pd.merge(rwy_mileFinal_complete, airports_declination, on = 'IDENT', how = 'left')

In [ ]:
rwy_mileFinal_linked.drop(columns = ['Column1', 'index_y'])

In [ ]:
airports_runways = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_runways_linked.csv')

In [ ]:
rwy_linked = pd.merge(rwy_mileFinal_linked, airports_runways, on = 'IDENT', how = 'left')

In [ ]:
rwy_linked

In [ ]:
rwy_linked.rename(columns = {'properties.DESIGNATOR': 'DESIGNATOR'}, inplace = True)

In [ ]:
# Split 'RWYLOCATION' field into the RWY and its designator so we can link to the properties.DESIGNATOR field in order
# to obtain the correct latitude/longitude coordinates for the appropriate runway
rwydesignator = []

for i in range(len(rwy_linked)):
    rwy_split = rwy_linked['RWYLOCATION'][i].split(' ')
    rwydesignator.append(rwy_split[1])

In [ ]:
# Add split column with RWY designators to dataframe as new field
rwy_linked['RWYDESIGNATOR'] = rwydesignator

In [ ]:
rwy_linked

#### Check if RWY designator found in REMARK matches one of the RWY designators from the rwy.csv dataset

*If YES: RWYinREMARK field is 'True'*

*If NO: RWYinREMARK field is 'False'*
***
***NOTE:*** Since we linked on the 'IDENT' field, there are repeated remarks as a result of there being multiple runways at
each airport. Thus, we are only interested in the records for which the 'RWYinREMARK' field is 'True'.

In [ ]:
rwy_linked['RWYinREMARK'] = rwy_linked.apply(lambda x: str(x.RWYDESIGNATOR) in str(x.DESIGNATOR), axis=1)

In [ ]:
other2158_runways98 = rwy_linked[rwy_linked['RWYinREMARK'] == True]

### Calculate Bearing Information for Runways

To do this, you take the number portion of the Runway Designator (in the 'RWYDESIGNATOR' field) and multiply it by 10 (effectively adding a 0 to the end) and account for Magnetic Variation (using the 'DECLINATION' field).

***Example:*** *RWY 32R has designator 32R which indicates a bearing of 320 degrees*

In [ ]:
other2158_runways98 = other2158_runways98.reset_index()

In [ ]:
designator = other2158_runways98['RWYDESIGNATOR']
designator_bearing = []

for i in range(len(other2158_runways98)):
    designator_bearing.append(int(re.sub("\D","",designator[i]))*10 + other2158_runways98['DECLINATION'][i])

In [ ]:
other2158_runways98['RWY_BEARING'] = designator_bearing

In [ ]:
other2158_runways98

### Convert Distances from NM to Kilometers

In [ ]:
# Convert all distance from NM to kilometers
dist_kilo = []

for i in range(len(other2158_runways98)):
    distanceKilo = int(re.sub("\D","",other2158_runways98['UAS_Distance'][i][0]))* 1.852 # converting NM to kilometers
    dist_kilo.append(distanceKilo)

In [ ]:
other2158_runways98['Distance_Kilometers'] = dist_kilo

# SWITCH Runway.Latitude and Runway.Longitude (I THINK ERIC HAS THESE MIXED UP ACCIDENTALLY)

In [ ]:
other2158_runways98.rename(columns = {'Runway.Latitude': 'Runway_Longitude', 'Runway.Longitude' : 'Runway_Latitude'}, inplace = True)

### Calculate UAS Latitude/Longitude Information using geopy library

In [ ]:
uas_lat = []
uas_long = []


for i in range(len(other2158_runways98)):
    lat_rwy = pd.to_numeric(other2158_runways98['Runway_Latitude'][i])
    long_rwy = pd.to_numeric(other2158_runways98['Runway_Longitude'][i])
    b = pd.to_numeric(other2158_runways98['RWY_BEARING'][i])
    d = pd.to_numeric(other2158_runways98['Distance_Kilometers'][i])

    origin = geopy.Point(lat_rwy, long_rwy)
    destination = geodesic(kilometers=d).destination(origin,b)

    lat2, lon2, = destination.latitude, destination.longitude

    uas_lat.append(lat2)
    uas_long.append(lon2)

# Append UAS Lat/Long information to DataFrame 
other2158_runways98['UAS_Latitude'] = uas_lat
other2158_runways98['UAS_Longitude'] = uas_long

In [ ]:
other2158_runways98

In [ ]:
# Export final new_df to .csv 
other2158_runways98.to_csv('rwy10_uasLatLong.csv', index = False)